# Bayesian Calibration with Gaussian Process Regression and MCMC Sampling: An Affine-Invariant Ensemble Approach with Gelman-Rubin Convergence Analysis

Load necessary modules

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, Matern, DotProduct, WhiteKernel
from sklearn.metrics import r2_score
import joblib
import emcee
import joblib
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.core.display import display, HTML
from matplotlib.lines import Line2D

/Users/phaniramasandeepchinta/Documents/Anaconda/anaconda3/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/var/folders/bz/tmghl8r56f58nx_wxdd36pv40000gp/T/ipykernel_14199/1508589538.py:12: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


Reading data and GPR fit

In [2]:
# Define the log-prior function
# The prior is defined as a uniform distribution between 0 and 1 for the parameters. 
# If the parameters (theta) are within this range, the log of the prior probability is 0 (since ln⁡(1)=0ln(1)=0).
# If the parameters are outside this range, the log of the prior probability is negative infinity (representing a probability of 0).

def lnprior(theta):
    if np.all(theta >= 0) and np.all(theta <= 1):
        return 0.0  # log(1) for uniform prior
    return -np.inf  # log(0)

In [3]:
# Define the log-likelihood function
# The likelihood is defined as a Gaussian likelihood centered at desired_nrmse 
# with a standard deviation of desired_std. This means you're assuming that the error (or discrepancy) 
# between the predicted NRMSE from your model and the desired NRMSE follows a normal distribution.
# The function returns the natural logarithm of this likelihood.

def lnlike(theta):
    predicted_nrmse = gpr_model.predict([theta])[0]
    # Gaussian likelihood centered at desired_nrmse with desired_std
    return -0.5 * ((predicted_nrmse - desired_nrmse) / desired_std)**2

In [ ]:
# Define the log-posterior function - Bayes Theorem
# A direct application of the logarithmic form of Bayes' theorem:
# ln⁡(Posterior)=ln⁡(Likelihood)+ln⁡(Prior)

def lnprob(theta):
    lp = lnprior(theta)
    if not np.isfinite(lp):
        return -np.inf
    return lp + lnlike(theta)

In [ ]:
# The Gelman-Rubin diagnostic, often represented as R_hat
# R_hat, is a widely used convergence diagnostic for Markov Chain Monte Carlo (MCMC) simulations. 
# It's based on comparing the variance between multiple independent chains to the variance within each chain. 
# The idea is that if all chains have converged to the target distribution, the between-chain and 
# within-chain variances should be similar. If R_hat is close to 1, it suggests that the chains have converged 
# to the target distribution. 
# We used convergence if R_hat < 1.1


def gelman_rubin(chains):
    """
    Compute the Gelman-Rubin diagnostic (R-hat) for MCMC chains.
    
    Parameters:
    - chains: MCMC chains, shape (n_chains, n_steps, n_parameters)
    
    Returns:
    - R_hat: Gelman-Rubin diagnostic for each parameter
    """
    n_chains, n_steps, n_parameters = chains.shape
    # Calculate chain means
    chain_means = np.mean(chains, axis=1)
    # Calculate overall mean
    overall_mean = np.mean(chains, axis=(0, 1))
    # Calculate between-chain variance (B/n)
    B_over_n = np.sum((chain_means - overall_mean)**2, axis=0) / (n_chains - 1)
    # Calculate within-chain variance (W)
    W = np.mean(np.var(chains, axis=1, ddof=1), axis=0)
    # Estimate of marginal posterior variance (var_hat plus correction)
    var_hat_plus = ((n_steps - 1) / n_steps) * W + B_over_n
    # Potential scale reduction factor (R-hat)
    R_hat = np.sqrt(var_hat_plus / W)
    
    return R_hat

In [ ]:
# Load output data from the specified file using numpy
output_test_data = np.loadtxt(f"codes/nrmse2/nrmse_train_d250t_12-17.csv", delimiter=",")

print(output_test_data.min())

In [ ]:
# Load the GPR model
gpr_model = joblib.load(f'codes/best_gpr_model_d250t_12-17.joblib')

In [ ]:
# Set desired NRMSE and standard deviation
# We would indeed aim for an NRMSE of zero, as this would indicate perfect agreement between 
# our model's predictions and the observed data. However, in practice, achieving an NRMSE of zero 
# might be unrealistic due to various sources of uncertainty (measurement errors, model structural errors, etc.).
# Experiment with different desired values of nrmse and std.

desired_nrmse = 0.4 #since minimum nrmse from the available samples is 0.403, it is better to take a slighlty lesser value 
desired_std = 0.02  # tried different combinations but the min nrmse was never going below 0.39; 

In [ ]:
# Set up the sampler
ndim, nwalkers = 5, 50
pos = [np.random.rand(ndim) for i in range(nwalkers)]
sampler = emcee.EnsembleSampler(nwalkers, ndim, lnprob)

# Run MCMC for some steps
nsteps = 20000
burnin = int(nsteps // 2)  # discard the first half of the steps
sampler.run_mcmc(pos, nsteps)

chains_burned = sampler.chain[:, burnin:, :]
chains_incl_burned = sampler.chain

R_hat_values = gelman_rubin(chains_burned)

# Check and print the R_hat values
for i, R_hat in enumerate(R_hat_values):
    print(f"Parameter {i+1}: R_hat = {R_hat:.3f}")
    if R_hat > 1.1:
        print(f"Warning: Parameter {i+1} might not have converged (R_hat = {R_hat:.3f}).")
        display(HTML('<span style="color: red;">Stopping because of non-convergence of one or more parameters</span>'))
        assert False

In [ ]:
#Save chains burned
joblib.dump(chains_burned, f'codes/saved_models/mcmc_samples_d250t_12-17.pkl')

In [ ]:
# Load the burned chains
chains_burned = joblib.load(f'codes/saved_models/mcmc_samples_d250t_12-17.pkl')

samples = chains_burned.reshape((-1, 5))  

In [ ]:
# Checking the parameter values over iterations
# Calculate the mean and standard deviation (or other spread measure) across walkers
mean_chains = np.mean(chains_incl_burned, axis=0)
std_chains = np.std(chains_incl_burned, axis=0)

# Number of parameters
num_params = mean_chains.shape[1]

# Create a larger figure for all subplots
plt.figure(figsize=(10, 4 * num_params))

for i in range(num_params):
    # Create subplot for each parameter
    plt.subplot(num_params, 1, i + 1)
    plt.plot(mean_chains[:, i], label=f'Mean Parameter {i+1}')
    plt.fill_between(range(mean_chains.shape[0]),
                     mean_chains[:, i] - std_chains[:, i],
                     mean_chains[:, i] + std_chains[:, i],
                     color='lightgrey', alpha=0.5, label=f'Std Dev Parameter {i+1}')
    plt.title(f'Summary Trace Plot for Parameter {i+1}')
    plt.xlabel('Step Number')
    plt.ylabel('Parameter Value')
    plt.ylim(0, 1)
    plt.legend()

# Save the entire figure
plt.tight_layout()  # Adjusts the plots to fit into the figure area
plt.savefig('summary_trace_plots.jpg')
plt.show()

In [ ]:
# Plots Posterior Distribution of parameters
# Parameter names and default values
params = ['$Q_{10}$', '$f_{CH_4}$', '$z_τ$', '$f_{D_0}$', '$K_{O_2}$']
defval = [0.2, 0.5, 0.57, 0.005, 0.091]

# Set global font to Times New Roman
plt.rcParams['font.family'] = 'Times New Roman'
plt.rcParams['font.size'] = 14

# Plotting the KDE for each parameter with the uniform prior
fig, axes = plt.subplots(2, 3, figsize=(12, 8))  # 2 rows, 3 columns

# Storing handles for legend entries
handles, labels = [], []

for i in range(5):
    row, col = divmod(i, 3)
    ax = axes[row, col]

    sns.kdeplot(samples[:, i], ax=ax, fill=True, color='skyblue', linewidth=2)
    # No need to store handle here, as we'll capture all handles later

    ax.axhline(y=1, color='green', linestyle='--')
    ax.axvline(x=defval[i], color='red', linestyle=':')

    # Adding 2-sigma confidence interval
    lower, upper = np.percentile(samples[:, i], [2.5, 97.5])
    ax.axvline(x=lower, color='purple', linestyle='-.')
    ax.axvline(x=upper, color='purple', linestyle='-.')

    ax.set_xlim(0, 1)
    ax.set_ylabel("Probability Density")
    ax.set_title(f"Parameter: {params[i]}")

# Hide the unused subplot
axes[1, 2].axis('off')

# Adding custom legend elements
from matplotlib.lines import Line2D
custom_lines = [Line2D([0], [0], color='skyblue', lw=4),
                Line2D([0], [0], color='green', linestyle='--'),
                Line2D([0], [0], color='red', linestyle=':'),
                Line2D([0], [0], color='purple', linestyle='-.')]

fig.legend(custom_lines, ["Posterior", "Uniform Prior", "Default Value", "2σ Interval"], 
           loc='center', bbox_to_anchor=(0.8, 0.3), fontsize=14)

# Set the xlabel for the last plot in each row
axes[0, 0].set_xlabel("Normalized Parameter Value", fontsize=14)
axes[0, 1].set_xlabel("Normalized Parameter Value", fontsize=14)
axes[0, 2].set_xlabel("Normalized Parameter Value", fontsize=14)
axes[1, 0].set_xlabel("Normalized Parameter Value", fontsize=14)
axes[1, 1].set_xlabel("Normalized Parameter Value", fontsize=14)

plt.tight_layout()
plt.savefig(f'pardis_d250t_12-17.png', bbox_inches='tight', dpi=300)
plt.show()


In [ ]:
# Draw parameter sets from the posterior distribution
n_samples = 50  # or however many you can afford to run
selected_samples = samples[np.random.choice(len(samples), n_samples, replace=False)]

# Convert to DataFrame
#df = pd.DataFrame(selected_samples)

# Save to CSV
#df.to_csv('selected_samples.csv', index=False)

# Run the E3SM Land Model with each parameter set and compute NRMSE for the validation data
nrmse_values = []
for sample in selected_samples:
    # Run the E3SM Land Model with 'sample' as the parameter set
    # Compare the model output to the validation data to compute NRMSE
    nrmse = gpr_model.predict(sample.reshape(1, -1))  # however you compute NRMSE
    nrmse_values.append(nrmse)

# Compute summary statistics for the NRMSE values
mean_nrmse = np.mean(nrmse_values)
lower_bound = np.percentile(nrmse_values, 2.5)
upper_bound = np.percentile(nrmse_values, 97.5)

print(f"Mean NRMSE: {mean_nrmse}")
print(f"2-sigma range: ({lower_bound}, {upper_bound})")
